In [1]:
import os

import geopandas as gpd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when

from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.utils.adapter import Adapter
from sedona.core.enums import GridType
from sedona.core.enums import IndexType
from sedona.core.spatialOperator import JoinQueryRaw

In [2]:
spark = SparkSession.builder.\
        master("local[*]").\
        appName("SedonaSQL-Example").\
        config("spark.serializer", KryoSerializer.getName).\
        config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
        config('spark.jars.packages',
               'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.1.0-incubating,'
               'org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
        getOrCreate()
SedonaRegistrator.registerAll(spark)
sc = spark.sparkContext
sc.setSystemProperty("sedona.global.charset", "utf8")

:: loading settings :: url = jar:file:/home/jovyan/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-985144e1-bc31-46ff-80b3-8b9036a2979c;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.1.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-core;2.12.2 in central
	found org.apache.sedona#sedona-core-3.0_2.12;1.1.0-incubating in central
	found org.apache.sedona#sedona-sql-3.0_2.12;1.1.0-incubating in central
	found org.datasyslab#geotools-wrapper;1.1.0-25.2 in central
:: resolution report :: resolve 893ms

In [3]:
census = ShapefileReader.readToGeometryRDD(sc, "data/drive-download-20220720T150603Z-001/census")
census_df = Adapter.toDf(census, spark)
census_df.createOrReplaceTempView("census")
census_df.printSchema()

subway = ShapefileReader.readToGeometryRDD(sc, "data/drive-download-20220720T150603Z-001/subway")
subway_df = Adapter.toDf(subway, spark)
subway_df.createOrReplaceTempView("subway")
subway_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- BLKID: string (nullable = true)
 |-- POPN_TOTAL: string (nullable = true)
 |-- POPN_WHITE: string (nullable = true)
 |-- POPN_BLACK: string (nullable = true)
 |-- POPN_NATIV: string (nullable = true)
 |-- POPN_ASIAN: string (nullable = true)
 |-- POPN_OTHER: string (nullable = true)
 |-- BORONAME: string (nullable = true)

root
 |-- geometry: geometry (nullable = true)
 |-- OBJECTID: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- ALT_NAME: string (nullable = true)
 |-- CROSS_ST: string (nullable = true)
 |-- LONG_NAME: string (nullable = true)
 |-- LABEL: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- NGHBHD: string (nullable = true)
 |-- ROUTES: string (nullable = true)
 |-- TRANSFERS: string (nullable = true)
 |-- COLOR: string (nullable = true)
 |-- EXPRESS: string (nullable = true)
 |-- CLOSED: string (nullable = true)



In [4]:
result = spark.sql("SELECT Sum(popn_total) FROM census")
result.show()

+-------------------------------+
|sum(CAST(popn_total AS DOUBLE))|
+-------------------------------+
|                      8175032.0|
+-------------------------------+



In [6]:
a1 = spark.sql("select distinct(c.geometry), popn_total from census c, subway s where st_distance(c.geometry, s.geometry)<500")
a1.createOrReplaceTempView("a1")
a2 = spark.sql("select sum(popn_total) from a1")
a3 = spark.sql("select geometry from a1")
a2.show()
a3.show()

22/07/26 06:26:23 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+-------------------------------+
|sum(CAST(popn_total AS DOUBLE))|
+-------------------------------+
|                      4415020.0|
+-------------------------------+



22/07/26 06:26:43 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+--------------------+
|            geometry|
+--------------------+
|POLYGON ((583655....|
|POLYGON ((592091....|
|POLYGON ((592869....|
|POLYGON ((596190....|
|POLYGON ((592282....|
|POLYGON ((591076....|
|POLYGON ((595367....|
|POLYGON ((596783....|
|POLYGON ((592024....|
|POLYGON ((591321....|
|POLYGON ((597596....|
|POLYGON ((592886....|
|POLYGON ((598710....|
|POLYGON ((596883....|
|POLYGON ((592447....|
|POLYGON ((585786....|
|POLYGON ((586040....|
|POLYGON ((583690....|
|POLYGON ((583484....|
|POLYGON ((584958....|
+--------------------+
only showing top 20 rows

